# **TR_2021/18 - Technical report: Case crossover for cardiovascular deaths and extreme events**


|Technical Report ID  |2021/18|
|--|--|
| Title |Case crossover for cardiovascular deaths and extreme events|
| Authors | Júlia De Lázari, Paula Dornhofer|
| Creation Date| 2021-08|


## Databases descriptions

**inputs:** 

- obitos_circulatorio.csv: Dataframe of deaths due to cardiovascular diseases from 2001 to 2019 (only data up to 2018 was used, to match the climate data).

- EV_VCP.csv: Dataframe with the extreme events computed. Viracopos data was used for this.

## Analysis

This report presents an analysis of the the _incidence rate ratio_ for the [extreme climate events](https://github.com/climate-and-health-datasci-Unicamp/project-climatic-variations-cardiovascular-diseases/blob/main/notebooks/TR_2020_05_Extreme_climatic_events_for_Campinas.ipynb) using a case crossover design.

##**Case crossover**

In a case-crossover study design, each person serves as his own control. The period immediately before the adverse outcome (death or hospitalization) is then compared with a period when no adverse outcome occurred [Gordis].

As the analyzed events are rare, we used a random number to generate the control for each patient to avoid bias. The number used was between 30 and 300. We considered a period of exposure of 5 days, i.e., we compared the exposure to extreme climatic events on the five days before the (case) with the exposure five days before the control.

Then, we performed a conditional logistic regression to obtain estimates of odds ratios (OR) and theirs 95 \% confidence interval. Because of the sampling strategy and the small initial risk (chance of a cardiovascular outcome in the city of Campinas), these ORs can be considered a reasonable estimate of the incidence rate ratio (IRR) [Davies].

Besides the extreme event of interest, meteorological parameters of maximum temperature, average pressure, and maximum humidity were included in all the regressions. Minimum temperature and humidity were not included due to the high correlation with the maximum temperature and humidity.

Because we used a random number to define the control, the results will be different each time. We used the mean value of 20 regressions to estimate the RR associated with each extreme event.

##**Import libraries**

In [1]:
#import python libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math
import statistics as stat
import scipy.stats as stats
import datetime as dt
import random

#plots
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
import seaborn as sns
import matplotlib.ticker as mticker

#array 
from array import array
from itertools import repeat

#logistic regression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

#files
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Load data**

###**Climatic data**

In [2]:
#-------------------------------------------------------------------#
#                          Load Viracopos data                      #
#-------------------------------------------------------------------#

#Load humidity dataframe
df_VCP = pd.read_csv('EV_VCP.csv')
df_VCP = df_VCP.drop(columns = {'Unnamed: 0'})
df_VCP['DATE'] = pd.to_datetime(df_VCP['DATE'])

In [3]:
#-------------------------------------------------------------------#
#                             Null rows                             #
#-------------------------------------------------------------------#
print("Percentage of null values VCP (2001-2018) \n")

print("TMIN:", round((len(df_VCP[df_VCP['TMIN'].isnull()]))/len(df_VCP)*100,2),"%")
print("TMAX:", round((len(df_VCP[df_VCP['TMAX'].isnull()]))/len(df_VCP)*100,2),"%")
print("AVGPRESSURE:", round((len(df_VCP[df_VCP['AVGPRESSURE'].isnull()]))/len(df_VCP)*100,2),"%")
print("HMIN:", round((len(df_VCP[df_VCP['HMIN'].isnull()]))/len(df_VCP)*100,2),"%")
print("HMAX:", round((len(df_VCP[df_VCP['HMAX'].isnull()]))/len(df_VCP)*100,2),"%")

print("\n")

print("Number of null rows \n")

print("TMIN:", len(df_VCP[df_VCP['TMIN'].isnull()]))
print("TMAX:", len(df_VCP[df_VCP['TMAX'].isnull()]))
print("AVGPRESSURE:",len(df_VCP[df_VCP['AVGPRESSURE'].isnull()]))
print("HMIN:", len(df_VCP[df_VCP['HMIN'].isnull()]))
print("HMAX:", len(df_VCP[df_VCP['HMAX'].isnull()]))

Percentage of null values VCP (2001-2018) 

TMIN: 0.18 %
TMAX: 0.18 %
AVGPRESSURE: 0.02 %
HMIN: 0.05 %
HMAX: 0.05 %


Number of null rows 

TMIN: 12
TMAX: 12
AVGPRESSURE: 1
HMIN: 3
HMAX: 3


In [4]:
#fill na with mean values
df_VCP['HMIN'].fillna(df_VCP['HMIN'].mean(), inplace=True)
df_VCP['HMAX'].fillna(df_VCP['HMAX'].mean(), inplace=True)
df_VCP['TMIN'].fillna(df_VCP['TMIN'].mean(), inplace=True)
df_VCP['TMAX'].fillna(df_VCP['TMAX'].mean(), inplace=True)
df_VCP['AVGPRESSURE'].fillna(df_VCP['AVGPRESSURE'].mean(), inplace=True)

###**Health data**

In [5]:
#-------------------------------------------------------------------#
#                      Circulatory deaths                           #
#-------------------------------------------------------------------#

df_obitos = pd.read_csv('obitos_circulatorio.csv')
df_obitos = df_obitos.drop(columns = {'Unnamed: 0','CODMUNRES','COMPLRES','CODMUNOCOR','LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII','CAUSABAS'}) #drop unneeded columns
df_obitos = df_obitos.rename(columns = {'DTOBITO':'DATE'}) #rename DTOBITO to DATE to merge dataframes
df_obitos = df_obitos[df_obitos['DATE']<='2018-12-31']
df_obitos = df_obitos[(df_obitos.DATE !='2000-02-29')&(df_obitos.DATE !='2004-02-29')&(df_obitos.DATE !='2008-02-29')&(df_obitos.DATE !='2012-02-29')&(df_obitos.DATE !='2016-02-29')] #remove leap year dates (02-29)
df_obitos = df_obitos.sort_values('DATE')

##**Functions**

In [6]:
def case_crossover_random_exposed(df_health,df_climate,event,show):

  #convert to datetime
  df_health['DATE'] = pd.to_datetime(df_health['DATE'])
  df_climate['DATE'] = pd.to_datetime(df_climate['DATE'])

  #exposition: 5 days before extreme event
  df_climate['exposed'] = df_climate[event].rolling(min_periods=1, window=5).sum()
  df_climate['exposed']= np.where(df_climate['exposed']==0,0,1)

  #cases 
  df_health1 = df_health.copy() #make a copy of health database
  df_health1['case'] = 1 #all the deaths are considered cases
  cases = pd.merge(df_health1, df_climate, on = 'DATE', how='outer') #merge with climate data
  
  #control - use a random number of days to determine the control
  df_health2 = df_health.copy() #make a copy of health database
  random_list = [] #create a random list with the size of the dataframe
  for i in range(0,len(df_health2)):
    random_list.append(random.randint(30, 300))
  #use a auxiliar column to shift dates
  df_health2['aux'] = random_list 
  df_health2['aux'] = df_health2['aux'].apply(lambda x: timedelta(days=x))
  df_health2['DATE'] = df_health2['DATE'] - df_health2['aux'] #shift dates
  df_health2['case']= 0 #all the shift dates are controls (a death didn't occured)
  control = pd.merge(df_health2, df_climate, on = 'DATE', how='outer') #merge with climate data

  #concat both dataframes
  df = pd.concat([cases, control])
  df = df[~df['case'].isnull()] #drop days without health data
  df = df[~df['exposed'].isnull()] #drop days without climate data
  df = df.sort_values('DATE') #sort by dates

  #cases and controls, exposed and unexposed
  case_exposed = len(df[(df['case']==1) & (df['exposed']==1)])
  case_unexposed = len(df[(df['case']==1) & (df['exposed']==0)])
  control_exposed = len(df[(df['case']==0) & (df['exposed']==1)])
  control_unexposed = len(df[(df['case']==0) & (df['exposed']==0)])
  
  #if show=True, print
  if show:
    print(f"case exposed: {case_exposed} \ncase unexposed: {case_unexposed} \ncontrol exposed: {control_exposed} \ncontrol unexposed: {control_unexposed} \n")

  #logistic regression - only with the event (exposition) as variable
  y = df['case']
  x = df['exposed']
  x = sm.add_constant(x)
  model = sm.Logit(y, x)
  result = model.fit(method='newton',disp=False)
  IRR = np.exp(result.params[1])
  
  #if show=True, print
  if show:
    print("IRR:", round(np.exp(result.params[1]),2))

  #return dataframe
  return df

In [7]:
def regression(data,event,variables):
        ###data - health data
        ###event - extreme event we want to compute the RR
        ###variables - explanatory variables

  #get the dataframe with cases and controls
  df = case_crossover_random_exposed(data, df_VCP,event,False)
  #y is the result - case or control
  y = df['case']
  #x are all of the explanotory variables
  x = df[variables]

  #perfom a logistic regression regression
  model = sm.Logit(y, x)
  result = model.fit(method='newton', disp=False)
  #rate ratio is the exponential of the coeficient associated to the extreme event
  RR = round(np.exp(result.params[0]),2)
  #confidence intervals
  ci = result.conf_int()
  lower_ci = round(np.exp(ci.iloc[0,0]),2)
  upper_ci = round(np.exp(ci.iloc[0,1]),2)
  
  #return the RR and CI
  return RR, lower_ci, upper_ci

In [8]:
def mean_rr(number,data,event,variables):
  ### number - times of regressions to compute the mean
  ### data - health database
  ###event - extreme event we want to compute the RR
  ###variables - explanatory variables

  #insert the exposition to the extreme event in the variables list
  variables.insert(0,'exposed')
  
  #create lists to store results
  irr_list = []
  lower_ci_list = []
  upper_ci_list = []

  #perform the regression n times
  for i in range(0, number+1):
    irr, lower_ci, upper_ci = regression(data,event,variables)
    irr_list.append(irr)
    lower_ci_list.append(lower_ci)
    upper_ci_list.append(upper_ci)

  #get the mean values
  print("IRR:",irr_list)
  print("IRR:", round(stat.mean(irr_list),2))
  print("lower ci:",lower_ci_list)
  print("upper ci:",upper_ci_list)
  print(f"CI: {round(stat.mean(lower_ci_list),2)} - {round(stat.mean(upper_ci_list),2)}")

## **Analysis**
Perform the regression 20 times and take the mean values

In [9]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'above_temp_range',columns)

IRR: [1.0, 1.0, 1.0, 1.0, 0.99, 0.99, 1.0, 1.01, 0.99, 1.0, 0.99, 1.01, 1.0, 0.99, 0.99, 0.99, 1.01, 1.01, 0.99, 1.0, 1.01]
IRR: 1.0
lower ci: [0.98, 0.98, 0.97, 0.98, 0.97, 0.96, 0.97, 0.98, 0.97, 0.97, 0.97, 0.98, 0.97, 0.97, 0.97, 0.97, 0.98, 0.98, 0.97, 0.98, 0.98]
upper ci: [1.03, 1.03, 1.03, 1.03, 1.02, 1.02, 1.02, 1.03, 1.02, 1.02, 1.02, 1.03, 1.02, 1.02, 1.02, 1.02, 1.03, 1.04, 1.02, 1.03, 1.03]
CI: 0.97 - 1.03


In [10]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'above_temp_dif',columns)

IRR: [1.01, 0.99, 0.94, 0.98, 0.99, 0.98, 0.96, 0.99, 0.95, 0.99, 0.95, 0.96, 0.98, 0.98, 0.98, 0.91, 0.98, 0.97, 0.96, 0.97, 0.99]
IRR: 0.97
lower ci: [0.96, 0.94, 0.9, 0.93, 0.94, 0.93, 0.91, 0.94, 0.91, 0.94, 0.9, 0.91, 0.93, 0.93, 0.93, 0.87, 0.93, 0.93, 0.91, 0.92, 0.94]
upper ci: [1.06, 1.04, 0.99, 1.03, 1.04, 1.03, 1.01, 1.04, 1.0, 1.04, 1.0, 1.01, 1.03, 1.03, 1.03, 0.96, 1.03, 1.02, 1.01, 1.02, 1.05]
CI: 0.92 - 1.02


In [11]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'LPW',columns)

IRR: [0.94, 0.96, 0.97, 0.96, 0.99, 0.98, 0.98, 0.99, 0.98, 0.99, 0.98, 0.98, 0.97, 0.95, 0.97, 1.0, 0.97, 0.94, 1.01, 0.97, 0.96]
IRR: 0.97
lower ci: [0.9, 0.92, 0.93, 0.92, 0.95, 0.94, 0.94, 0.95, 0.93, 0.95, 0.94, 0.94, 0.93, 0.91, 0.93, 0.96, 0.92, 0.9, 0.97, 0.93, 0.92]
upper ci: [0.99, 1.0, 1.01, 1.0, 1.03, 1.03, 1.02, 1.04, 1.02, 1.04, 1.03, 1.03, 1.02, 1.0, 1.02, 1.05, 1.01, 0.98, 1.06, 1.01, 1.01]
CI: 0.93 - 1.02


In [12]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'HPW',columns)

IRR: [1.01, 0.99, 1.03, 1.02, 0.99, 0.97, 0.99, 0.98, 1.03, 1.0, 1.0, 1.02, 1.01, 0.99, 1.01, 0.98, 1.0, 1.0, 0.99, 1.04, 1.01]
IRR: 1.0
lower ci: [0.97, 0.95, 0.99, 0.98, 0.96, 0.94, 0.96, 0.95, 0.99, 0.96, 0.96, 0.98, 0.97, 0.96, 0.97, 0.94, 0.97, 0.97, 0.95, 1.0, 0.97]
upper ci: [1.05, 1.03, 1.07, 1.06, 1.03, 1.01, 1.03, 1.02, 1.07, 1.03, 1.04, 1.06, 1.04, 1.03, 1.05, 1.02, 1.04, 1.04, 1.03, 1.08, 1.05]
CI: 0.97 - 1.04


In [13]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'above_pressure_dif',columns)

IRR: [0.99, 0.98, 1.0, 0.98, 0.99, 0.98, 0.99, 0.98, 0.98, 0.98, 1.0, 0.98, 0.99, 0.98, 0.98, 0.99, 0.99, 0.99, 0.99, 0.99, 0.98]
IRR: 0.99
lower ci: [0.96, 0.95, 0.97, 0.95, 0.97, 0.95, 0.96, 0.95, 0.96, 0.96, 0.98, 0.96, 0.96, 0.96, 0.96, 0.97, 0.97, 0.96, 0.96, 0.97, 0.96]
upper ci: [1.01, 1.0, 1.02, 1.0, 1.02, 1.0, 1.01, 1.0, 1.01, 1.01, 1.03, 1.01, 1.01, 1.01, 1.01, 1.02, 1.02, 1.01, 1.01, 1.02, 1.01]
CI: 0.96 - 1.01


In [14]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'LHW',columns)

IRR: [1.04, 1.05, 1.02, 1.02, 1.06, 1.06, 1.06, 1.07, 1.09, 1.03, 1.04, 1.05, 1.11, 1.04, 1.06, 1.07, 1.04, 1.07, 1.06, 1.08, 1.05]
IRR: 1.06
lower ci: [0.98, 0.99, 0.97, 0.96, 1.0, 1.0, 0.99, 1.01, 1.03, 0.97, 0.98, 0.99, 1.05, 0.98, 1.0, 1.0, 0.98, 1.0, 1.0, 1.02, 0.99]
upper ci: [1.1, 1.12, 1.09, 1.08, 1.12, 1.12, 1.12, 1.14, 1.16, 1.09, 1.1, 1.11, 1.18, 1.11, 1.13, 1.13, 1.11, 1.13, 1.12, 1.14, 1.11]
CI: 0.99 - 1.12


In [15]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'HHW',columns)

IRR: [1.01, 0.99, 1.06, 1.02, 0.99, 1.01, 1.01, 1.05, 1.03, 0.97, 1.03, 1.03, 1.04, 1.0, 1.0, 1.03, 1.0, 0.98, 0.98, 1.01, 1.02]
IRR: 1.01
lower ci: [0.96, 0.93, 1.0, 0.96, 0.94, 0.95, 0.95, 0.99, 0.97, 0.92, 0.97, 0.97, 0.99, 0.94, 0.94, 0.97, 0.95, 0.93, 0.93, 0.95, 0.96]
upper ci: [1.07, 1.04, 1.13, 1.08, 1.05, 1.07, 1.07, 1.11, 1.09, 1.03, 1.09, 1.09, 1.11, 1.06, 1.05, 1.09, 1.06, 1.04, 1.04, 1.07, 1.08]
CI: 0.96 - 1.07


In [16]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'above_humidity_range',columns)

IRR: [0.99, 0.99, 0.99, 0.98, 0.98, 1.01, 1.0, 0.99, 1.0, 0.99, 0.99, 1.0, 1.0, 0.99, 1.0, 0.99, 1.01, 1.0, 1.0, 1.01, 0.99]
IRR: 1.0
lower ci: [0.96, 0.97, 0.96, 0.96, 0.96, 0.98, 0.98, 0.97, 0.98, 0.97, 0.97, 0.97, 0.97, 0.97, 0.98, 0.97, 0.99, 0.97, 0.98, 0.98, 0.96]
upper ci: [1.01, 1.02, 1.01, 1.0, 1.01, 1.03, 1.03, 1.02, 1.03, 1.02, 1.02, 1.02, 1.02, 1.02, 1.03, 1.01, 1.04, 1.02, 1.03, 1.03, 1.01]
CI: 0.97 - 1.02


In [17]:
columns = ['TMAX','AVGPRESSURE','HMAX']
mean_rr(20,df_obitos,'above_humidity_dif',columns)

IRR: [1.0, 1.01, 1.02, 0.98, 0.98, 1.01, 0.96, 1.01, 1.0, 1.0, 1.06, 1.04, 1.03, 0.97, 1.0, 1.01, 1.03, 0.99, 1.01, 1.03, 0.99]
IRR: 1.01
lower ci: [0.95, 0.96, 0.97, 0.93, 0.93, 0.96, 0.91, 0.96, 0.95, 0.95, 1.0, 0.99, 0.98, 0.92, 0.95, 0.96, 0.98, 0.94, 0.96, 0.98, 0.94]
upper ci: [1.05, 1.06, 1.08, 1.03, 1.03, 1.06, 1.01, 1.06, 1.05, 1.06, 1.11, 1.09, 1.09, 1.01, 1.05, 1.06, 1.09, 1.04, 1.06, 1.09, 1.04]
CI: 0.96 - 1.06


##**References**

L. Gordis, “Epidemiology, Saunders Elsevier", Philadelphia, Pa, USA, 2013.

H. T. O. Davies, I. K. Crombie, and M. Tavakoli, “When can odds ratios mislead?”Bmj, vol.316, no. 7136, pp. 989–991, 1998.